In [271]:
import pandas as pd
from sqlalchemy import create_engine, URL
import psycopg2 as pg
import panel as pn
import datetime as dt

In [255]:
# Dados da minha base de dados
dbname = "Trabalho"
schema = "sistema"
host = "localhost"
port = "5432"
user = "postgres"
password = "qwe123"

# Conexão com banco de dados - psycopg2
conpg = pg.connect(dbname = dbname, host = host, port = port, user = user, password = password)

# Alterando para o esquema que uso no pgadmin
cursor = conpg.cursor()
cursor.execute(f"set schema '{schema}'")

# Conexão com banco de dados - SqlAlchemy
# Criando a URL
url_con = URL.create(
    "postgresql+psycopg2", username = user, password = password, host = host, database = dbname
)

# Setando para o schema no bd
engine = create_engine(url_con, connect_args = {'options': '-c search_path={}'.format(schema)})

In [256]:
#Configurando o Panel
pn.extension()
pn.extension("tabulator")
pn.extension(notificatons = True)

# Inserindo um Paciente

In [257]:
# Funcões de tratamento das entradas

def tratarCpf():
    valor = cpf.value
    if len(valor) < 11:
        raise Exception("CPF incompleto")
    if not (valor.isdecimal()):
        raise Exception("CPF tem que conter só números decimais")

def tratarTamanho(string):
    valor = string
    if len(valor) > 50:
        raise Exception("A palavra tem que ter menos de 50 letras")

def tratarPNome():
    valor = pNome.value
    tratarTamanho(valor)
    
def tratarUNome():
    valor = uNome.value
    tratarTamanho(valor)

def tratarGenero():
    valor = genero.value
    if valor == "Masculino":
        valor = "M"
    elif valor == "Feminino":
        valor = "F"
    elif valor == "Outro":
        valor = "O"
    elif valor == "Não informar":
        valor = "-"
    else:
        raise Exception("Gênero inválido")
    return valor

def tratarRua():
    valor = rua.value
    tratarTamanho(valor)

def tratarNumCasa():
    valor = numCasa.value
    if len(valor) > 5:
        raise Exception("Número da casa muito grande")

def tratarComplemento():
    valor = complemento.value
    if len(valor) > 10:
        raise Exception("Complemento da casa muito grande")

def tratarBairro():
    valor = bairro.value
    tratarTamanho(valor)

def tratarCidade():
    valor = cidade.value
    tratarTamanho(valor)

def tratarEstado():
    valor = estado.value
    tratarTamanho(valor)

def tratarPais():
    valor = pais.value
    tratarTamanho(valor)


def tratarEntradas():
    tratarCpf()
    tratarPNome()
    tratarUNome()
    tratarGenero()
    tratarRua()
    tratarNumCasa()
    tratarComplemento()
    tratarBairro()
    tratarCidade()
    tratarEstado()
    tratarPais()

In [258]:
# Criando váriavel para campos vazios
campo_vazio = ""

# Criando os componentes da tela

# Componentes para entidade 'Pessoa'
cpf = pn.widgets.TextInput(name = "CPF", value = "85167924120", placeholder = "Informe seu CPF", disabled = False)
pNome = pn.widgets.TextInput(name = "Primeiro Nome", value = "Eduarda", placeholder = "Informe seu primeiro nome", disabled = False)
uNome = pn.widgets.TextInput(name = "Último Nome", value = "Josefa Duarte", placeholder = "Informe seu último nome ", disabled = False)
genero = pn.widgets.RadioBoxGroup(name = "Gênero", options = ["Masculino", "Feminino", "Outro", "Não informar"], disabled = False)
rua = pn.widgets.TextInput(name = "Rua", value = "Rua Rio Grande do Norte", placeholder = "Informe sua rua", disabled = False)
numCasa = pn.widgets.TextInput(name = "Número da casa", value = "140", placeholder = "Informe o número da casa ", disabled = False)
complemento = pn.widgets.TextInput(name = "Complemento", value = "", placeholder = "Informe o complemento da casa", disabled = False)
bairro = pn.widgets.TextInput(name = "Bairro", value = "Conjunto Universitário", placeholder = "Informe seu bairro", disabled = False)
cidade = pn.widgets.TextInput(name = "Cidade", value = "Rio Branco", placeholder = "Informe sua cidade", disabled = False)
estado = pn.widgets.TextInput(name = "Estado", value = "AC", placeholder = "Informe seu estado", disabled = False)
pais = pn.widgets.TextInput(name = "País", value = "Brasil", placeholder = "Informe seu país", disabled = False)
dataNasc = pn.widgets.DatePicker(name = "Data de nascimento", disabled = False)

# Criando os botões de inserção e consulta de dados
botaoInserir = pn.widgets.Button(name = "Inserir", button_type = "primary")
botaoDados = pn.widgets.Button(name = "Dados", button_type = "primary")

def retornarPaciente():
    # Consulta os dados da entidade Paciente
    try:
        sql = "select * from Paciente order by id_paciente"
        dadosRetornados = pd.read_sql_query(sql, engine)
        tabela = pn.widgets.Tabulator(dadosRetornados)
        return tabela
    except:
        cursor.close()
        return pn.pane.Alert("Erro ao mostrar os dados")

def inserir_paciente():
    try:
        cursor = conpg.cursor()

        #Trata as entradas dos componentes
        try:
            tratarEntradas()
        except Exception as e:
            return pn.pane.Alert(f"Erro nas entradas. Error: {e}")
        else:
            # Ajustando o genero para o fortamto do bd
            generoFormatado = tratarGenero()
            # Criando a entidade Pessoa
            try:
                sqlPessoa = "insert into Pessoa(cpf, pnome, unome, genero, numero_casa, complemento, rua, bairro, cidade, estado, pais, data_nasc) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(sqlPessoa, (cpf.value, pNome.value, uNome.value, generoFormatado, numCasa.value,complemento.value, rua.value, bairro.value, cidade.value, estado.value, pais.value, dataNasc.value))
            except pg.Error as e:
                conpg.rollback()
                cursor.close()
                return pn.pane.Alert(f"Erro em Pessoa. Error: {e.pgerror}")
    
            # Criando a entidade Paciente
            try:
                sqlPaciente = "insert into Paciente(id_paciente, paciente_cpf) values ((select max(id_paciente) + 1 from Paciente ), %s)"
                cursor.execute(sqlPaciente, (cpf.value,))
            except pg.Error as e:
                conpg.rollback()
                return pn.pane.Alert(f"Erro em Paciente. Error: {e.pgerror}")
    except pg.Error as e:
        conpg.rollback()
        cursor.close()
        return pn.pane.Alert(f"Erro no backend. Error: {e}")
    else:
        # Salvando mudanças no BD
        conpg.commit()
        return retornarPaciente()

# Definindo as funções para os botões
def criador_tabela1(inserir, dados):
    if dados:
        return retornarPaciente()
    if inserir:
        return inserir_paciente()

# Associando as funções para os botões
associarBotao1 = pn.bind(criador_tabela1, botaoInserir, botaoDados)


In [272]:
pn.Row(
    pn.Column("Inserir Paciente", cpf, pNome, uNome, genero, rua, numCasa, complemento, bairro, cidade, estado, pais, dataNasc,
              pn.Row(botaoInserir),
              pn.Row(botaoDados)              
            ),
    pn.Row(associarBotao1)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'733ac394-c93f-468a-85fe-4078386ac369': {'version…

RuntimeError: Models must be owned by only a single document, ImportedStyleSheet(id='9f2666c6-3af5-42ab-8380-3695af87fb42', ...) is already in a doc

# Cadastrando uma consulta entre médico e paciente

In [260]:
def retornarMedicos():
    try:
        sql = "select p.cpf from Pessoa p join Medico m on p.cpf = m.medico_cpf order by p.cpf asc"
        dadosRetornados = pd.read_sql_query(sql, engine)
        dadosList = dadosRetornados.to_dict('list')
        medicos = dadosList['cpf']
    except Exception as e:
        print(f"Error ao consultar o nome dos médicos. Erro: {e}")
    else:
        return medicos
    
def retornarPacientes():
    try:
        sql = "select pe.cpf from Pessoa pe join Paciente pa on pe.cpf = pa.paciente_cpf order by pe.cpf asc"
        dadosRetornados = pd.read_sql_query(sql, engine)
        dadosList = dadosRetornados.to_dict('list')
        pacientes = dadosList['cpf']
    except Exception as e:
        print(f"Error ao consultar o nome dos Pacientes. Erro: {e}")
    else:
        return pacientes
        
def retornarIdMedico(cpf):
    try:
        sqlIdMedico = f"Select m.id_medico from Medico m join Pessoa p on m.medico_cpf = p.cpf where p.cpf = '{cpf}'"
        dados = pd.read_sql_query(sqlIdMedico, engine)
        idMedico = dados['id_medico'][0]
    except pg.Error as e:
        raise Exception(f"Erro ao retornar o id do médico. Erro: {e}")
    else:
        return int(idMedico)
        

def retornarIdPaciente(cpf):
    try:
        sqlIdPaciente = f"Select pa.id_paciente from Pessoa pe join Paciente pa on pe.cpf = pa.paciente_cpf where pe.cpf = '{cpf}'"
        dados = pd.read_sql_query(sqlIdPaciente, engine)
        idPaciente = dados['id_paciente'][0]
    except pg.Error as e:
        raise Exception(f"Erro ao retornar o id do paciente. Erro: {e}")
    else:
        return int(idPaciente)

In [261]:
# Consultar os nomes pro autocomplete
cpfMedicos = []
cpfPacientes = []

cpfMedicos = retornarMedicos()
cpfPacientes = retornarPacientes()

# Declarando componentes
nomeClinica = pn.widgets.TextInput(name = "Nome da Clínica", value = "Clínica A", disabled = False, placeholder = "Insira o nome da clínica")
cpfMedico =  pn.widgets.AutocompleteInput(name = "CPF do Médico", value = "47355794272 ", disabled = False, placeholder = "Insirar o cpf do médico(a)", options = cpfMedicos, case_sensitive = False, search_strategy = "includes")
cpfPaciente = pn.widgets.AutocompleteInput(name = "CPF do Paciente", value = "85167924120", disabled = False, placeholder = "Insirar o cpf do paciente", options = cpfPacientes, case_sensitive = False, search_strategy = "includes")
dataConsulta = pn.widgets.DatePicker(name = "Data da consulta", disabled = False)
horaConsulta = pn.widgets.TimePicker(name = "Hora da consulta", disabled = False, format = "H:i", clock = "24h")
motivo = pn.widgets.TextInput(name = "Motivo da consulta", value = "Consulta", disabled = False, placeholder = "Qual motivo da consulta")
observacoes = pn.widgets.TextInput(name = "Observações", value = "", disabled = False, placeholder = "Observações")

cadastrarConsulta = pn.widgets.Button(name = "Marcar consulta", button_type = "primary")

    
# Criando a função que retorna todas as consultas

def retornarConsulta():
    try:
        sqlConsulta = "select * from Consulta order by id_consulta asc"
        dados = pd.read_sql_query(sqlConsulta, engine)
        tabela = pn.widgets.Tabulator(dados)
        return tabela
    except pg.Error as e:
        return pn.pane.Alert(f"Erro ao retornar as consultas. Erro: {e.pgerror}")

# Criando função de cadastro
def criarConsulta():
    idMedico = retornarIdMedico(cpfMedico.value)
    idPaciente = retornarIdPaciente(cpfPaciente.value)
    
    
    def traduzirDiaSemana(diasTrabalho):
        dias = []
        diasTrabalhoFormatado = diasTrabalho.split(', ')
                
        for dia in diasTrabalhoFormatado:
            if dia == "seg":
                dias.append(0)
            elif dia == "ter":
                dias.append(1)
            elif dia == "qua":
                dias.append(2)
            elif dia == "qui":
                dias.append(3)
            elif dia == "sex":
                dias.append(4)
            elif dia == "sab":
                dias.append(5)
            else:
                raise Exception("Todo mundo precisa descansar. É domingo!")
        return dias
    
    def verificaDias(dado):
        conteudo = dado[0]
        data = dataConsulta.value
        dias = traduzirDiaSemana(conteudo)
        
        trabalha = False
        for dia in dias:
            if data.weekday() == dia:
                trabalha = True
        return trabalha

        
        
    def verificaTurno(dado):
        conteudo = dado[0]
        hora = int(horaConsulta.value.hour())
        
        if (conteudo == 'M' and hora < 12) or (conteudo == 'V' and (hora > 13 and hora < 18)):
            # Médico trabalha no turno
            return True
        else:
            return False

    def verificarDisponibilidade():
        # Verificar turno e dias de trabalho do médico
        
        sqlDados = f"select j.dias, j.turno from Jornada_trabalho j join Medico m on m.id_jornada_trabalho = j.id_jornada_trabalho where m.id_medico = {idMedico}"
        dados = pd.read_sql_query(sqlDados, engine)
        dadosFormatados = dados.to_dict('list')
        
        # Retorna uma 'string' em ambos que ainda estão em <List>
        diasMedico = dadosFormatados['dias']
        turnoMedico = dadosFormatados['turno']
        
        d = verificaDias(diasMedico)
        t = verificaTurno(turnoMedico)
        
        if d == True and t == True:
            # Verificar se está vago
            sqlDados = f"select c.data_consulta, c.hora_consulta from Consulta c join Medico m on c.id_medico = m.id_medico where c.id_medico = {idMedico}"
            dados = pd.read_sql_query(sqlDados, engine)
            
            data = dataConsulta.value
            hora = horaConsulta.value
            
            for valor in dados.values.tolist():
                dtDb = valor[0]
                hrDB = valor[1]
                if dtDb == data and hrDB == hora:
                    raise Exception('Data e hora já marcados')
        else:
            if d == False: return Exception("Médico não trabalha nesse dia")
            if t == False: return Exception('Médico não trabalha nesse turno')
        

    try:
        'text'
        #verificarDisponibilidade()
    except pg.Error as e:
        conpg.rollback()
        return pn.pane.Alert(f"Erro ao consulta data e hora da consulta. Erro: {e.pgerror}")
    except Exception as e:
        conpg.rollback()
        return pn.pane.Alert(f"Erro ao cadastrar consulta (try 1). Erro: {e}")
    else:
        try:
            ultimoId = "(select max(id_consulta) + 1 from Consulta)"
            sqlCadConsulta = f"""INSERT INTO Consulta(id_consulta, nome_clinica, id_medico, id_paciente, data_consulta, hora_consulta, motivo, observacoes) VALUES ({ultimoId}, %s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(sqlCadConsulta, ( nomeClinica.value, idMedico, idPaciente, dataConsulta.value, horaConsulta.value, motivo.value, observacoes.value, ))
        except pg.Error as e:
            conpg.rollback()
            cursor.execute("ROLLBACK")
            return pn.pane.Alert(f"Erro ao cadastrar consulta (try 2). Erro: {e.pgerror}")   
        except Exception as e:
            return pn.pane.Alert(f"Erro: {e}")     
        else:
            conpg.commit()
            return retornarConsulta()
        
# Definindo as funções para o botão

def associarBotao(inserir):
    if inserir:
        return criarConsulta()
    
botaoAssociadoConsulta = pn.bind(associarBotao, cadastrarConsulta)

In [262]:
pn.Column(
    'Cadastrar Consulta',
    nomeClinica,
    pn.Row(cpfMedico, cpfPaciente),
    pn.Row(dataConsulta, horaConsulta),
    pn.Row(motivo, observacoes),
    cadastrarConsulta,
    botaoAssociadoConsulta,
    sizing_mode = "stretch_both", scroll = True
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'90d257e7-153e-4e17-865e-4450748f08c9': {'version…

KeyError: 0

# Vários tipos de consultas que poderiam ser feitas

#### Retornar todos os pacientes que um médico já atendeu

In [267]:
# Pegar o nome de todos médicos
cpfMedicos = []
cpfMedicos = retornarMedicos()

cpfMedico =  pn.widgets.AutocompleteInput(name = "CPF do médico", disabled = False, placeholder = "Insira o cpf do médico(a)", options = cpfMedicos, case_sensitive = False, search_strategy = "includes")
especialidade = pn.widgets.TextInput(name = "Especialidade", disabled = True, placeholder = "A especialidade do médico é...")
crm = pn.widgets.TextInput(name = "CRM", disabled = True, placeholder = "O CRM do médico é...")
uf_crm = pn.widgets.TextInput(name = "UF do CRM", disabled = True, placeholder = "A UF do CRM do médico é...")


consultarPacientes = pn.widgets.Button(name = "Consultar pacientes atendidos", button_type = "primary")


def atualizarComponentes(evento):
    idMedico = retornarIdMedico(evento.new)
        
    try:
        sqlDados = f"select especialidade, crm, uf from Medico m where id_medico = {idMedico}"
        dados = pd.read_sql_query(sqlDados, engine)        
    except pg.Error as e:
        conpg.rollback()
        return pn.pane.Alert(f"Erro ao retornar os dados. Erro: {e.pgerror}")
    else:
        dadosFormatados = dados.to_dict('list')
        
        especialidade.value = str(dadosFormatados['especialidade'][0])
        crm.value = str(dadosFormatados['crm'][0])
        uf_crm.value = str(dadosFormatados['uf'][0])
        
    
# "Assistidor" dos componentes
watcher = cpfMedico.param.watch(atualizarComponentes, ['value'], onlychanged = True)
    
    
    


def consultarPacienteBD():
    idMedico = retornarIdMedico(cpfMedico.value)
    
    try:
        sqlDados = f"""select pe.pnome, pe.unome, pe.genero, to_char(pe.data_nasc, 'dd/mm/yyyy') as "data de nascimento", h.tipo_sanguineo, h.rh_sanguineo, d.descricao, to_char(c.data_consulta, 'dd/mm/yyyy') as "data da consulta"
from pessoa pe inner join paciente pa on pe.cpf = pa.paciente_cpf
inner join historico_paciente h on h.id_paciente = pa.id_paciente
inner join doencas d on h.id_historico_paciente = d.id_historico_paciente
inner join consulta c on c.id_paciente = h.id_paciente
where c.id_medico = {idMedico}"""
        dados = pd.read_sql_query(sqlDados, engine)
    except pg.Error as e:
        conpg.rollback()
        return pn.pane.Alert(f"Erro ao retornar os pacientes. Erro: {e.pgerror}")
    else:
        return pn.widgets.Tabulator(dados)    





# Criar função para o butão
def criador_tabela2(consultar):
    if consultar:
        return consultarPacienteBD()
    
associarBotao2 = pn.bind(criador_tabela2, consultarPacientes)

In [273]:
pn.Column(
    'Pacientes atendidos',
    pn.Row(cpfMedico, especialidade),
    pn.Row(crm, uf_crm),
    pn.Row(consultarPacientes),
    associarBotao2,
    sizing_mode = "stretch_both"
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2ed096fa-7a06-4f58-8535-c5303b426710': {'version…

RuntimeError: Models must be owned by only a single document, ImportedStyleSheet(id='03641b60-ae55-4b14-9baf-67575d2ddd4c', ...) is already in a doc

#### Retornar todas informações de um paciente

In [265]:
# Pegar o nome de todos médicos
cpfPacientes = []
try:
    sqlDados = "select pe.cpf from Pessoa pe join Paciente pa on pe.cpf = pa.paciente_cpf order by cpf asc"
    dados = pd.read_sql_query(sqlDados, engine)
except pg.Error as e:
    cursor.execute("ROLLBACK")
    deuMerda = pn.pane.Alert(f"Erro ao retornar os cpf's. Erro: {e.pgerror}")
else:

    dadosFormatados = dados.to_dict('list')
    cpfPacientes = dadosFormatados['cpf']
    
try:
    sqlDados = """select pe.cpf, pe.pnome as "nome", pe.unome as "sobrenome", to_char(pe.data_nasc, 'dd/mm/yyyy') as "data de nascimento", pe.cidade, pe.estado from Pessoa pe join Paciente pa
on pe.cpf = pa.paciente_cpf order by pe.pnome asc"""
    dados = pd.read_sql_query(sqlDados, engine)
except pg.Error as e:
    conpg.rollback()
    infoPacientes = pn.pane.Alert(f"Erro ao exibir os pacientes para o componente. Erro: {e.pgerror}")
else:
    infoPacientes = pn.widgets.Tabulator(dados)

    cpfPaciente =  pn.widgets.AutocompleteInput(name = "CPF do paciente", disabled = False, placeholder = "Insirar o CPF do Paciente", options = cpfPacientes, case_sensitive = False, search_strategy = "includes")

    consultarPaciente = pn.widgets.Button(name = "Consultar paciente", button_type = "primary")

    def consultarPacienteBD():
        idPaciente = -1
        
        try:
            sqlDados = f"select id_paciente from paciente where paciente_cpf = '{cpfPaciente.value}'"
            cursor.execute(sqlDados)
        except pg.Error as e:
            cursor.execute("ROLLBACK")
            return pn.pane.Alert(f"Erro ao retornar o id do paciente. Erro: {e.pgerror}")
        else:
            idPaciente = cursor.fetchone()[0]
            
        try:
            sqlDados = f"""select e.diagnosticos, e.medicamentos, e.resultados, a.descricao as "alergias", d.descricao as "doenças", c.data_consulta, c.motivo
                from historico_paciente h join paciente p on h.id_paciente = p.id_paciente
                join exame e on e.id_historico_paciente = h.id_historico_paciente
                join alergias a on a.id_historico_paciente = h.id_historico_paciente
                join doencas d on d.id_historico_paciente = h.id_historico_paciente
                join consulta c on c.id_paciente = p.id_paciente
                where p.id_paciente = {idPaciente};"""
            dados = pd.read_sql_query(sqlDados, engine)
        except pg.Error as e:
            return pn.pane.Alert(f"Erro ao retornar os dados do paciente. Erro: {e.pgerror}")
        else:
            return pn.widgets.Tabulator(dados)    


    # Criar função para o butão
    def criador_tabela3(consultar):
        if consultar:
            return consultarPacienteBD()
        
    associarBotao3 = pn.bind(criador_tabela3, consultarPaciente)

In [266]:
pn.Column(
    "Tabela de Pacientes",
    infoPacientes, cpfPaciente, consultarPaciente,
    associarBotao3,
    sizing_mode = "scale_both", scroll = True
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'43f24510-363e-438d-83d8-86dcf0013ce2': {'version…